In [2]:
import pandas as pd
signal = pd.read_excel("first.train.xlsx")

## Function to Calculate Average Localisation Error

In [3]:
def euclidean_distance(coord1, coord2):
    return np.linalg.norm(coord1 - coord2)

def average_localization_error(predicted_coords, ground_truth_coords):
    errors = []
    for pred_coord, gt_coord in zip(predicted_coords, ground_truth_coords):
        error = euclidean_distance(pred_coord, gt_coord)
        errors.append(error)
    return np.mean(errors)

## Preparing the data to fit into CNN Model

In [4]:
signal.describe()

,WAP001,WAP002,WAP003,WAP004,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,...,WAP618,WAP619,WAP620,id,timestamp,x,y,f,Labels,Actual_labels
count,8640.000000,8640.000000,8640.000000,8640.000000,8640.000000,8640.000000,8640.000000,8640.000000,8640.000000,8640.000000,...,8640.0,8640.0,8640.0,8.640000e+03,8.640000e+03,8640.000000,8640.000000,8640.000000,8640.000000,8640.000000
mean,-7.038310,-3.175926,2.209722,10.110880,59.010764,-7.260532,4.066088,8.981134,10.331944,58.375926,...,100.0,100.0,100.0,1.081049e+08,2.016062e+16,8.524312,22.949487,4.000000,24.514583,23.514583
std,89.807832,88.172132,88.163900,91.235399,77.150028,89.780622,88.206010,91.256607,90.218637,77.588284,...,0.0,0.0,0.0,4.320745e+06,6.685720e+09,3.584252,4.097289,1.000058,13.842608,13.842608
min,-97.000000,-97.000000,-95.000000,-95.000000,-97.000000,-98.000000,-95.000000,-95.000000,-98.000000,-96.000000,...,100.0,100.0,100.0,1.011001e+08,2.016060e+16,4.134772,16.699815,3.000000,1.000000,0.000000
25%,-85.000000,-82.000000,-80.000000,-83.000000,100.000000,-85.000000,-79.000000,-83.000000,-82.000000,100.000000,...,100.0,100.0,100.0,1.041073e+08,2.016061e+16,4.134772,18.487919,3.000000,13.000000,12.000000
50%,-75.000000,-64.000000,-62.000000,100.000000,100.000000,-75.000000,-61.000000,100.000000,100.000000,100.000000,...,100.0,100.0,100.0,1.081049e+08,2.016062e+16,8.524312,22.064128,4.000000,24.500000,23.500000
75%,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,100.0,100.0,100.0,1.121024e+08,2.016062e+16,12.913852,25.640336,5.000000,36.250000,35.250000
max,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,100.0,100.0,100.0,1.151096e+08,2.016062e+16,12.913852,29.216544,5.000000,48.000000,47.000000


In [49]:
signal_labels

,x,y
0,12.913852,29.216544
1,12.913852,29.216544
2,12.913852,29.216544
3,12.913852,29.216544
4,12.913852,29.216544
...,...,...
8635,12.913852,29.216544
8636,12.913852,29.216544
8637,12.913852,29.216544
8638,12.913852,29.216544


In [50]:
signal_shape = signal.shape
signal_labels_shape = signal_labels.shape

In [51]:
signal_shape

(8640, 623)

In [52]:
signal_labels_shape

(8640, 2)

## Reshaping the Data

In [53]:
import numpy as np
signal_array = signal.values
reshaped_signal = signal_array.reshape(signal_shape[0], signal_shape[1], 1)

signal_labels_array = signal_labels.values
reshaped_signal_labels = signal_labels_array.reshape(signal_labels_shape[0], signal_labels_shape[1], 1)

## Normalizing the data

In [54]:
from sklearn.preprocessing import MinMaxScaler
flattened_signal = reshaped_signal.reshape(-1, 1)
scaler = MinMaxScaler()
normalized_flattened_signal = scaler.fit_transform(flattened_signal)
normalized_signal = normalized_flattened_signal.reshape(reshaped_signal.shape)

flattened_signal_labels = reshaped_signal_labels.reshape(-1, 1)
normalized_flattened_signal_labels = scaler.fit_transform(flattened_signal_labels)
normalized_signal_labels = normalized_flattened_signal_labels.reshape(reshaped_signal_labels.shape)

In [55]:
normalized_signal.shape

(8640, 623, 1)

In [56]:
normalized_signal_labels.shape

(8640, 2, 1)

## Creating CNN Model

In [57]:
import tensorflow as tf
from tensorflow.keras import layers, models

#defining the cnn model
def create_cnn_model(input_shape):
    model = models.Sequential()
    model.add(layers.Conv1D(32, kernel_size=3, activation = 'relu', input_shape=input_shape[1: ]))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Conv1D(64, kernel_size=3, activation='relu'))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(2, activation='linear'))

    return model

input_shape = (8640, 623, 1)

model = create_cnn_model(input_shape)

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

model.summary()

model.fit(normalized_signal, normalized_signal_labels, epochs=10, batch_size=32, validation_split=0.2)

C:\Users\wmd04\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                      │ (None, 621, 32)             │             128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (None, 310, 32)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ (None, 308, 64)             │           6,208 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_1 (MaxPooling1D)       │ (None, 154, 64)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 9856)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       1,261,696 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 2)                   │             258 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,268,290 (4.84 MB)

 Trainable params: 1,268,290 (4.84 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 10s 33ms/step - loss: 0.0599 - mae: 0.1819 - val_loss: 0.0248 - val_mae: 0.1327
Epoch 2/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 7s 32ms/step - loss: 0.0246 - mae: 0.1345 - val_loss: 0.0240 - val_mae: 0.1329
Epoch 3/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 6s 29ms/step - loss: 0.0243 - mae: 0.1342 - val_loss: 0.0238 - val_mae: 0.1301
Epoch 4/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 6s 29ms/step - loss: 0.0245 - mae: 0.1336 - val_loss: 0.0238 - val_mae: 0.1327
Epoch 5/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 6s 29ms/step - loss: 0.0240 - mae: 0.1325 - val_loss: 0.0237 - val_mae: 0.1307
Epoch 6/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 7s 30ms/step - loss: 0.0246 - mae: 0.1344 - val_loss: 0.0286 - val_mae: 0.1415
Epoch 7/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 6s 28ms/step - loss: 0.0241 - mae: 0.1313 - val_loss: 0.0241 - val_mae: 0.1315
Epoch 8/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 6s 28ms/step - loss: 0.0241 - mae: 0.1324 - val_loss: 0.0237 - val_mae: 0.1297
Epoch 9/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 6s 28ms

In [58]:
some_data = normalized_signal[:10]

In [59]:
some_labels = normalized_signal_labels[:10]

In [60]:
prepared_data = np.reshape(some_data, (some_data.shape[0], some_data.shape[1], 1))

In [61]:
predictions = model.predict(prepared_data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 209ms/step


In [62]:
predictions

array([[0.17557274, 0.7729285 ],
       [0.17557274, 0.7729285 ],
       [0.17557274, 0.7729285 ],
       [0.17557274, 0.7729285 ],
       [0.17557274, 0.7729285 ],
       [0.17557274, 0.7729285 ],
       [0.17557274, 0.7729285 ],
       [0.17557274, 0.7729285 ],
       [0.17557274, 0.7729285 ],
       [0.17557274, 0.7729285 ]], dtype=float32)

In [63]:
some_labels.tolist()

[[[0.3500183295661009], [1.0]],
 [[0.3500183295661009], [1.0]],
 [[0.3500183295661009], [1.0]],
 [[0.3500183295661009], [1.0]],
 [[0.3500183295661009], [1.0]],
 [[0.3500183295661009], [1.0]],
 [[0.17500916478305042], [1.0]],
 [[0.17500916478305042], [1.0]],
 [[0.17500916478305042], [1.0]],
 [[0.17500916478305042], [1.0]]]

## Evaluating the Model

In [64]:
from sklearn.metrics import mean_squared_error
import numpy as np

predictions = model.predict(normalized_signal)

if normalized_signal_labels.ndim > 2:
    normalized_signal_labels = normalized_signal_labels.squeeze()
if predictions.ndim > 2:
    predictions = predictions.squeeze()

mse = mean_squared_error(normalized_signal_labels, predictions)
rmse = np.sqrt(mse)

270/270 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step


In [65]:
mse

0.023810588969916474

In [66]:
rmse

0.15430680143764394

## Testing the Model

In [67]:
import pandas as pd
strat_test_set = pd.read_excel("test.twentyfive.xlsx")

In [68]:
# strat_test_set = strat_test_set.drop("Label", axis=1)

In [69]:
strat_test_set.shape

(7392, 625)

In [70]:
x_test = strat_test_set.drop(['x', 'y'], axis = 1) #dropping columns 'x' and 'y' from testing data frame
y_test = strat_test_set[['x', 'y']].copy() #selecting only columns 'x' and 'y' from testing data frame

In [71]:
x_test_shape = x_test.shape
y_test_shape = y_test.shape

In [72]:
# x_test_shape

In [73]:
# y_test_shape

In [74]:
#Reshaping the test data
x_test_array = x_test.values
reshaped_x_test = x_test_array.reshape(x_test_shape[0], x_test_shape[1], 1)

y_test_array = y_test.values
reshaped_y_test = y_test_array.reshape(y_test_shape[0], y_test_shape[1], 1)

In [75]:
#Normalizing the test data
from sklearn.preprocessing import MinMaxScaler
flattened_x_test = reshaped_x_test.reshape(-1, 1)
scaler = MinMaxScaler()
normalized_flattened_x_test = scaler.fit_transform(flattened_x_test)
normalized_x_test = normalized_flattened_x_test.reshape(reshaped_x_test.shape)

flattened_y_test = reshaped_y_test.reshape(-1, 1)
normalized_flattened_y_test = scaler.fit_transform(flattened_y_test)
normalized_y_test = normalized_flattened_y_test.reshape(reshaped_y_test.shape)

In [76]:
# normalized_x_test.shape

In [77]:
# normalized_y_test.shape

In [78]:
from sklearn.metrics import mean_squared_error
import numpy as np

predictions = model.predict(normalized_x_test)

if normalized_y_test.ndim > 2:
    normalized_y_test = normalized_y_test.squeeze()
if predictions.ndim > 2:
    predictions = predictions.squeeze()

mse = mean_squared_error(normalized_y_test, predictions)
rmse = np.sqrt(mse)

231/231 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step


In [79]:
# rmse

In [80]:
# mse

In [81]:
final_predictions = model.predict(normalized_x_test)
final_mse = mean_squared_error(normalized_y_test, final_predictions)
final_rmse = np.sqrt(mse)
predicted_coords = final_predictions
ground_truth_coords = normalized_y_test.tolist()
avg_error = average_localization_error(predicted_coords, ground_truth_coords)
print("Average Localisation Error:", avg_error)
print("RMSE:", final_rmse) 
# print(final_predictions, normalized_y_test.tolist())

231/231 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step
Average Localisation Error: 0.18973636435638866
RMSE: 0.1440058253364601
